# Day 6: Chronal Coordinates 

The device on your wrist beeps several times, and once again you feel like you're falling."Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."

The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.

## If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the **largest distance** from the other points.

Using only the *Manhattan distance*, **determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate** (and aren't tied in distance to any other coordinate).

Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:

    1, 1
    1, 6
    8, 3
    3, 4
    5, 5
    8, 9

If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:

    ..........
    .A........
    ..........
    ........C.
    ...D......
    .....E....
    .B........
    ..........
    ..........
    ........F.

This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:

    aaaaa.cccc
    aAaaa.cccc
    aaaddecccc
    aadddeccCc
    ..dDdeeccc
    bb.deEeecc
    bBb.eeee..
    bbb.eeefff
    bbb.eeffff
    bbb.ffffFf

Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.

In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

What is the size of the largest area that isn't infinite?


In [1]:
import re, math
input_data = open('input.txt').read().split('\n')
suggested_locations = {}
for i,line in enumerate(input_data):
    try:
        x,y = re.findall(r'-?\d+',line)
        point = (int(x),int(y))
        tag = chr(i+65) if i < 26 else chr(i+71)
        suggested_locations[point] = {}
        suggested_locations[point]['Tag'] = tag
    except ValueError:
        print(f'No pattern matches found on line {i}: {line}')

No pattern matches found on line 50: 


In [2]:
def plot_point_field(point_field):
    plot = {}
    
    x = [point[0] for point in point_field]
    min_x, max_x = min(x), max(x)
    y = [point[1] for point in point_field]
    min_y, max_y = min(y), max(y)
    
    for y in range(min_y, max_y+1):
        for x in range(min_x, max_x+1):
            plot[(x,y)] = {}
            if (x,y) in point_field:
                plot[(x,y)]['Tag'] = point_field[(x,y)]['Tag']
            else:
                tag = '.'
                plot[(x,y)]['Tag'] = tag
    return plot

In [3]:
def get_closest_point_tag(point,target_points):
    x,y = point
    closest_point = '.'
    min_distance = math.inf
    
    if (x,y) in target_points:
        return target_points[point]['Tag']
    
    for point in target_points:
        distance = abs(point[0] - x) + abs(point[1] - y)
        if distance == min_distance:
            closest_point = '!'
        elif distance < min_distance:
            closest_point = target_points[point]['Tag']
            min_distance = distance
    return closest_point

In [4]:
plot = plot_point_field(suggested_locations)
for point in plot:
    plot[point]['Tag'] = get_closest_point_tag(point,suggested_locations)
    
x = [point[0] for point in plot]
min_x, max_x = min(x), max(x)
y = [point[1] for point in plot]
min_y, max_y = min(y), max(y)

possible_tags = set([suggested_locations[point]['Tag'] for point in suggested_locations])
counts = { tag : 0 for tag in possible_tags }

for point in plot:
    if plot[point]['Tag'] in possible_tags:
        if point[0] == min_x or point[0] == max_x or point[1] == min_y or point[1] == max_y:
            possible_tags.remove(plot[point]['Tag'])
            counts[plot[point]['Tag']] = - math.inf
        
        counts[plot[point]['Tag']] += 1

max_c = ('',0)
for loc in counts:
    if counts[loc] > max_c[1]:
        max_c = (loc,counts[loc])
        
print(f'Location {max_c[0]} largest area {max_c[1]}')

Location a largest area 5035


#### --- Part Two ---

On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region **near as many coordinates as possible.**

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:

    ..........
    .A........
    ..........
    ...###..C.
    ..#D###...
    ..###E#...
    .B.###....
    ..........
    ..........
    ........F.

In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:

    Distance to coordinate A: abs(4-1) + abs(3-1) =  5
    Distance to coordinate B: abs(4-1) + abs(3-6) =  6
    Distance to coordinate C: abs(4-8) + abs(3-3) =  4
    Distance to coordinate D: abs(4-3) + abs(3-4) =  2
    Distance to coordinate E: abs(4-5) + abs(3-5) =  3
    Distance to coordinate F: abs(4-8) + abs(3-9) = 10
    Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30

Because the total distance to all coordinates (30) is less than 32, the location is within the region.

This region, which also includes coordinates D and E, has a total size of 16.

Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.

What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?


In [6]:
for point in plot:
    distance_sum = 0
    for loc in suggested_locations:
        distance_sum += abs(loc[0] - point[0]) + abs(loc[1] - point[1])
    plot[point]['Distance Sum'] = distance_sum

In [7]:
count = 0
for point in plot:
    if plot[point]['Distance Sum'] < 10000:
        count += 1
count

35294